In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import math
from matplotlib.backends.backend_pdf import PdfPages
import openpyxl
import pprint
import itertools
import pickle
import os

In [2]:
# データ読み出し関数

def sarch_data(start, end, data) :
    return data[start:end]

# 時間データを整数値に変更
def time_to_int(time_data) :
#    print(type(time_data))
    time_data.as_integer_ratio()
    time_data *= 25000
#    print(time_data)
    time_data = int(time_data)
    return time_data

def import_time(rb, name, column_point) :
    rb_sheet = rb[name]
    sig_ori = [rb_sheet.cell(column=column_point, row=i).value for i in range(4, 68)]
    return sig_ori

  
        
# データ書き込み関数

def write_data(wb_sheet, sig, column_point) :
    wb_sheet.cell(column=column_point, row=1, value="volt")
    for i, sig_data in enumerate(sig) :
        #print(sig_data)
        wb_sheet.cell(column=column_point, row=i+2, value=sig_data)
        
        
        
def time_to_ripple(df, rb, rb_sheet_name, rb_column_point, wb, wb_column_point) :
    time_data = import_time(rb, rb_sheet_name, rb_column_point)
    for i in range(int(len(time_data)/2)-1) :
        sheet_new = wb.create_sheet()
    #print(time_data)
    sheet_point = 0
    for i, sheet_name in enumerate(wb.sheetnames) :
        start = time_to_int(time_data[2*i])
        #print(start)
        end = time_to_int(time_data[(2*i)+1])
        #print(i, sheet_name)
        ripple_sig = sarch_data(start, end, df)
        ripple_sig = ripple_sig.values.tolist()
        ripple_sig = list(itertools.chain.from_iterable(ripple_sig))
        write_data(wb[sheet_name], ripple_sig, 1)
        

def make_individual_data(df, rb, rb_sheet_name, rb_start_column, save_path, episode) :
    for i in range(rb_start_column, rb_start_column+4) :
        wb = openpyxl.Workbook()
        time_to_ripple(df, rb, rb_sheet_name, i, wb, 1)
        wb.save(save_path + str(rb[rb_sheet_name].cell(column=rb_start_column, row=2).value) + "-" + episode + "-" + str(rb[rb_sheet_name].cell(column=rb_start_column+1, row=2).value).replace("H", "") + str((i-rb_start_column)+2) + ".xlsx")
        wb.close()  

        
# ディレクトリ作成
def my_makedirs(path):
    if not os.path.isdir(path):
        os.makedirs(path)


In [3]:
rb = openpyxl.load_workbook("ripple.xlsx")
pattern_dict1 = {}
sumpsth = 0

#print(rb.sheetnames)
# シート名指定
rb_sheet_name = "restraint 抽出"

# 経験指定
episode = "restraint"

# 個体指定
individual = "B39-HR"

# 列の開始位置指定
start_column = 21

# パス指定
save_path = "/home/nodoka/win2/bp-ripple-data/" + episode +"/" + individual +"/"

# 活動電位データ読み込み
with open('/home/nodoka/win2/bp-filter-data/' + episode + '/' + individual  + '.pickle', mode='rb') as fp:
    df = pickle.load(fp)
    
my_makedirs(save_path)
    
make_individual_data(df, rb, rb_sheet_name, start_column, save_path, episode)